**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The agent above is called an *epsilon-greedy algorithm*. In such agent, `epsilon` is used to ensure that the agent sometimes randomly explores the environement instead of always acting optimally (according to itself). Choosing the right `epsilon` is quite complex and environment-dependent; this problem is referred to as the *exploration-exploitation trade-off*. Exploration is needed to guarantee that the agent is provided enough information about the global environment's states and rewards.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0: # right
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: # left
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2: # 
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: # 
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        state = self.get_state()

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        return self.get_state()
    
    def get_state(self):
        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=1 # set small when debugging
epochs_test=1 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

`position` contains information of the edges of the island (-1) and of the last occupied position of the agent (1).

`board` contains information of all the rewards in the environment (either boni or mali).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()

    def learned_act(self, s):
        action = np.random.randint(0,self.n_action)
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state,train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.0/10.0. Average score (-1.0)
Final score: -1.0


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




By definition we have
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{0\leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]
\end{equation*}
ie
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[r(s,a) + \gamma\sum_{1\leq t}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]
\end{equation*}
which results in
\begin{equation*}Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}
Applying this last property to $Q^{\pi^*}=Q^*$ gives:
\begin{equation*}Q^*(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{*}(s',a')]
\end{equation*}
and, given that
\begin{equation*}Q^*(s,\pi^*(s))=\max_{a}Q^*(s,a)
\end{equation*}
we obtain
\begin{equation*}Q^*(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')] \> .
\end{equation*}

Now, to create an objective function, $r+\gamma\max_{a'}Q(s',a',\theta)$ can be viewed as a target value for $Q(s,a,\theta)$; the difference between the two is called a temporal difference. At every state and action, this difference is minimal with value 0 when $\pi=\pi^*$. Then, setting
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}
as the loss function to optimize is a reasonable choice. This loss function was introduced by https://arxiv.org/pdf/1312.5602.pdf to solve Atari games with deep reinforcement learning.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        index = np.random.randint(0,len(self.memory))
        return self.memory[index]
    
    def size(self):
        return len(self.memory)

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        
    def learned_act(self, s):
        q = self.model.predict(np.array([s,]))[0]
        a = np.argmax(q)
        return a

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        if self.memory.size() == 10*self.batch_size:
            print("Now starting to learn from collected episodes")
        
        if self.memory.size() >= 10*self.batch_size:
            input_states = np.zeros((self.batch_size, 5,5,self.n_state))
            target_q = np.zeros((self.batch_size, 4))

            for i in range(self.batch_size):
                ######## FILL IN
                s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
                target_q_ = self.model.predict(np.array([s_,]))[0]
                if game_over_:
                    ######## FILL IN
                    target_q_a = r_
                else:
                    ######## FILL IN
                    target_q_a = r_ + self.discount*np.max(self.model.predict(np.array([n_s_,]))[0])
                target_q_[a_] = target_q_a
                target_q[i,:] = target_q_[:]
                input_states[i,:,:,:] = s_
            ######## FILL IN
            # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
            target_q = np.clip(target_q, -3, 3)

            l = self.model.train_on_batch(input_states, target_q)
        else:
            l = 0

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Reshape((50,),input_shape=(5,5,self.n_state)))
        model.add(Dense(32))
        model.add(Activation('elu'))
        model.add(Dense(16))
        model.add(Activation('elu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        #model.predict(np.zeros((2,5,5,self.n_state))) # dummy call to init input_shape
        self.model = model
        

In [12]:
epochs_train=31 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.03, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/031 | Loss 0.0000 | Win/lose count 2.0/12.0 (-10.0)
Now starting to learn from collected episodes
Epoch 001/031 | Loss 0.0221 | Win/lose count 0/4.0 (-4.0)
Epoch 002/031 | Loss 0.0140 | Win/lose count 4.5/6.0 (-1.5)
Epoch 003/031 | Loss 0.0063 | Win/lose count 2.5/5.0 (-2.5)
Epoch 004/031 | Loss 0.0212 | Win/lose count 4.0/7.0 (-3.0)
Epoch 005/031 | Loss 0.0115 | Win/lose count 1.5/2.0 (-0.5)
Epoch 006/031 | Loss 0.0113 | Win/lose count 2.0/5.0 (-3.0)
Epoch 007/031 | Loss 0.0081 | Win/lose count 6.0/3.0 (3.0)
Epoch 008/031 | Loss 0.0082 | Win/lose count 2.0/3.0 (-1.0)
Epoch 009/031 | Loss 0.0330 | Win/lose count 3.0/5.0 (-2.0)
Epoch 010/031 | Loss 0.0128 | Win/lose count 2.0/8.0 (-6.0)
Epoch 011/031 | Loss 0.0178 | Win/lose count 3.0/3.0 (0.0)
Epoch 012/031 | Loss 0.0189 | Win/lose count 2.5/8.0 (-5.5)
Epoch 013/031 | Loss 0.0024 | Win/lose count 2.5/0 (2.5)
Epoch 014/031 | Loss 0.0090 | Win/lose count 3.5/4.0 (-0.5)
Epoch 015/031 | Loss 0.0169 | Win/lose count 8.0/5.0 (3.0)


In [13]:
print(agent.model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1632      
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 68        
Total params: 2,228
Trainable params: 2,228
Non-trainable params: 0
_________________________________________________________________
None


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [14]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super().__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(filters=8, kernel_size=2, strides=(1, 1), padding='valid',activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=1, padding='valid'))
        model.add(Conv2D(filters=16, kernel_size=2, strides=(1, 1), padding='valid',activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=1, padding='valid'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [15]:
epochs_train=31 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.03, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/031 | Loss 0.0000 | Win/lose count 1.5/1.0 (0.5)
Now starting to learn from collected episodes
Epoch 001/031 | Loss 0.0193 | Win/lose count 3.0/7.0 (-4.0)
Epoch 002/031 | Loss 0.0026 | Win/lose count 1.0/7.0 (-6.0)
Epoch 003/031 | Loss 0.0179 | Win/lose count 0.5/4.0 (-3.5)
Epoch 004/031 | Loss 0.0230 | Win/lose count 2.0/2.0 (0.0)
Epoch 005/031 | Loss 0.0017 | Win/lose count 2.5/7.0 (-4.5)
Epoch 006/031 | Loss 0.0162 | Win/lose count 2.5/2.0 (0.5)
Epoch 007/031 | Loss 0.0044 | Win/lose count 4.5/2.0 (2.5)
Epoch 008/031 | Loss 0.0170 | Win/lose count 3.5/3.0 (0.5)
Epoch 009/031 | Loss 0.0088 | Win/lose count 4.0/0 (4.0)
Epoch 010/031 | Loss 0.0102 | Win/lose count 3.0/2.0 (1.0)
Epoch 011/031 | Loss 0.0143 | Win/lose count 3.5/2.0 (1.5)
Epoch 012/031 | Loss 0.0175 | Win/lose count 2.5/3.0 (-0.5)
Epoch 013/031 | Loss 0.0009 | Win/lose count 1.5/2.0 (-0.5)
Epoch 014/031 | Loss 0.0015 | Win/lose count 3.0/2.0 (1.0)
Epoch 015/031 | Loss 0.0028 | Win/lose count 4.0/1.0 (3.0)
Epoch 

In [16]:
print(agent.model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 4, 4, 8)           72        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 3, 3, 8)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 16)          528       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 68        
Total params: 668
Trainable params: 668
Non-trainable params: 0
_________________________________________________________________
None


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

epochs_test = 20
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.0/0. Average score (2.0)
Win/lose count 1.5/0. Average score (1.75)
Win/lose count 6.5/0. Average score (3.3333333333333335)
Win/lose count 4.0/2.0. Average score (3.0)
Win/lose count 5.0/1.0. Average score (3.2)
Win/lose count 0/0. Average score (2.6666666666666665)
Win/lose count 2.5/0. Average score (2.642857142857143)
Win/lose count 4.5/1.0. Average score (2.75)
Win/lose count 0.5/0. Average score (2.5)
Win/lose count 0.5/0. Average score (2.3)
Win/lose count 1.5/0. Average score (2.227272727272727)
Win/lose count 0.5/0. Average score (2.0833333333333335)
Win/lose count 3.0/0. Average score (2.1538461538461537)
Win/lose count 0/0. Average score (2.0)
Win/lose count 4.0/0. Average score (2.1333333333333333)
Win/lose count 1.5/0. Average score (2.09375)
Win/lose count 0.5/0. Average score (2.0)
Win/lose count 1.5/0. Average score (1.9722222222222223)
Win/lose count 4.0/2.0. Average score (1.9736842105263157)
Win/lose count 4.5/0. Average score (2.1)
F

In [18]:
HTML(display_videos('cnn_test10.mp4'))

In [19]:
HTML(display_videos('fc_test10.mp4'))

We tested several setting for the environment by changing the temperature. We find that:
- by comparing the average reward over 20 episodes, the CNN agent outperforms FC in every environment setting, although it has less parameters
- in most of the episodes, the agents (both FC and CNN) soon enter a 2-step cycle left/right or up/down; as a result, they gather very few boni and mali
- the 2 agents take a bit more steps to enter such cycle when the temperature is closer to 1 than to 0.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [20]:
def epsilon_linear(e,epsilon_range,last_epoch):
    """
    Inputs: e (int) an epoch iteration
            epsilon range (tuple of 2 int) epsilon at e = 0 and epsilon at e = last_epoch
            last_epoch (int)
    Outputs: epsilon
    """
    if e <= last_epoch:
        epsilon = (1/last_epoch)*((last_epoch - e)*epsilon_range[0] + e*epsilon_range[1])
    else:
        epsilon = epsilon_range[1]
    return epsilon

def train_explore(agent,env,epoch,epsilon_setter,prefix=''):
    """A remake of function train with 2 diff:
    - call `env.step(action,train=True)` instead of `env.step(action)` (Assumes env is of class `EnvironmentExploring`)
    - updates `agent.epsilon` at each epoch according to `epsilon_setter`
    
    Inputs:
        epsilon_setter (function) takes 1 int argument and returns 1 scalar between 0 and 1
        env (class) should implement EnvironmentExploring
        other inputs (agent,epoch,prefix) are the same as function train    
    """
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        
        agent.set_epsilon(epsilon_setter(e))

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action,True) # MODIFIED from function train

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    """
    A remake of class Environment. Adds a penalization when visiting an state already visited during current episode.
    Also adds a layer of features to the state observation corresponding to the visited states."""
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring,self).__init__(grid_size, max_time, temperature)
        
        self.malus_dejavu = 0.1
        
    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        
        state, reward, game_over = super(EnvironmentExploring,self).act(action)

        if train:
            reward += -self.malus_position[self.x, self.y]
        
        if self.malus_position[self.x,self.y] != self.malus_dejavu:
            self.malus_position[self.x, self.y] = self.malus_dejavu
            state = self.get_state()

        return state, reward, game_over
    
    def get_state(self):
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
    def reset(self):
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        
        return super(EnvironmentExploring,self).reset()

In [21]:
# Training
epochs_train = 51

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
epsilon_setter = lambda e: epsilon_linear(e,[0.1,0.],30)
train_explore(agent, env, epochs_train,epsilon_setter, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/051 | Loss 0.0000 | Win/lose count 4.0/25.500000000000107 (-21.500000000000107)
Now starting to learn from collected episodes
Epoch 001/051 | Loss 0.0071 | Win/lose count 6.0/25.800000000000033 (-19.800000000000033)
Epoch 002/051 | Loss 0.0136 | Win/lose count 1.0/20.000000000000014 (-19.000000000000014)
Epoch 003/051 | Loss 0.0086 | Win/lose count 1.0/20.900000000000027 (-19.900000000000027)
Epoch 004/051 | Loss 0.0198 | Win/lose count 13.0/19.200000000000014 (-6.2000000000000135)
Epoch 005/051 | Loss 0.0066 | Win/lose count 8.0/21.500000000000043 (-13.500000000000043)
Epoch 006/051 | Loss 0.0043 | Win/lose count 3.0/20.70000000000003 (-17.70000000000003)
Epoch 007/051 | Loss 0.0115 | Win/lose count 7.0/24.40000000000006 (-17.40000000000006)
Epoch 008/051 | Loss 0.0067 | Win/lose count 4.0/20.200000000000028 (-16.200000000000028)
Epoch 009/051 | Loss 0.0499 | Win/lose count 10.0/24.700000000000085 (-14.700000000000085)
Epoch 010/051 | Loss 0.0037 | Win/lose count 4.5/20.9000

In [22]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 5.5/2.0. Average score (3.5)
Win/lose count 9.5/3.0. Average score (5.0)
Win/lose count 1.5/2.0. Average score (3.1666666666666665)
Win/lose count 4.0/0. Average score (3.375)
Win/lose count 5.0/2.0. Average score (3.3)
Win/lose count 6.0/2.0. Average score (3.4166666666666665)
Win/lose count 7.5/2.0. Average score (3.7142857142857144)
Win/lose count 2.0/0. Average score (3.5)
Win/lose count 7.0/3.0. Average score (3.5555555555555554)
Win/lose count 4.0/0. Average score (3.6)
Win/lose count 3.5/0. Average score (3.590909090909091)
Win/lose count 2.0/2.0. Average score (3.2916666666666665)
Win/lose count 4.5/2.0. Average score (3.230769230769231)
Win/lose count 0.5/0. Average score (3.0357142857142856)
Win/lose count 0.5/1.0. Average score (2.8)
Win/lose count 5.0/4.0. Average score (2.6875)
Win/lose count 4.0/2.0. Average score (2.6470588235294117)
Win/lose count 8.0/3.0. Average score (2.7777777777777777)
Win/lose count 1.5/0. Average score (2.710526315789474)
Win/lose 

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

Ideas to test out:
- would the previous architecture benefit from more training epochs? Could we get an optimal agent with this architecture (maybe tuning `epsilon`, sgd learning rate and sgd momentum ...)?
- how well does the previous agent generalize to environments with other temperatures?
- how accurate is the finally learned Q-function?